### P452 - Computational Physics - MidTerms
##### Name: Jyotirmaya Shivottam
##### Roll: 1711069

In [1]:
import numpy as np

#### Q4

In [2]:
def lin_reg(x, y, yerr, ret_all=False):
    """
    Linear Regression Module

    Parameters
    ----------
    x: array_like
    y: array_like
    yerr: array_like
    ret_all: bool

    """
    # Calculating relevant quantities
    S =  np.sum(1 / (yerr**2))
    Sx = np.sum(x / (yerr**2))
    Sy = np.sum(y / (yerr**2))
    Sxx = np.sum((x**2) / (yerr**2))
    Syy = np.sum((y**2) / (yerr**2))
    Sxy = np.sum((x*y) / (yerr**2))

    delta = S * Sxx - Sx ** 2
    c = (Sxx * Sy - Sx * Sxy) / delta
    m = (S * Sxy - Sx * Sy) / delta
    yfit = m * x + c

    omega_2m = S / delta
    omega_2c = Sxx / delta
    
    cov = -Sx / delta
    r2 = Sxy / (Sxx * Syy)

    if ret_all:
        return yfit, m, c, omega_2m, omega_2c, cov, r2
    else: # Returning only the things required for fitting
        return yfit, m, c

def chi_sq(yexp, yfit, yerr):
    """
    Calculates Goodness-of-fit (Chi-Square)

    Parameters
    ----------
    yexp: array_like
    yfit: array_like
    yerr: array_like

    """
    chi_sq = np.sum(((yexp - yfit) / yerr) ** 2)
    chi_sqn = chi_sq / (yexp.shape[0] - 2)

    return chi_sq, chi_sqn

In [3]:
# from LibPython.Library import Statistics
# s = Statistics()

dat = np.genfromtxt("msfit.txt")
dat

array([[  1., 106.,  10.],
       [ 15.,  80.,   9.],
       [ 30.,  98.,  10.],
       [ 45.,  75.,   9.],
       [ 60.,  74.,   8.],
       [ 75.,  73.,   8.],
       [ 90.,  49.,   7.],
       [105.,  38.,   6.],
       [120.,  37.,   6.],
       [135.,  22.,   5.]])

In [4]:
time = dat[:, 0]
counts = dat[:, 1]
std_count = dat[:, 2]

counts = np.log(counts)
std_count = 1 / std_count

In [5]:
fit, m, _, sig2m, _, _, _ = lin_reg(time, counts, std_count, True)

In [6]:
halflife = np.log(2) / (-m)
error = np.sqrt(sig2m)

print(f"Half life: {halflife}\nError: {error}")

Half life: 74.8766130697403
Error: 0.0010146470187917449


In [7]:
chi_sq_, chi_sqn = chi_sq(counts, fit, std_count)
print(f"Obtained value: {chi_sq_} < 16.919 (for DoF = 10 - 1 = 9). So, the null hypothesis is satisfied.") # Source: https://people.richland.edu/james/lecture/m170/tbl-chi.html

Obtained value: 15.17084059456294 < 16.919 (for DoF = 10 - 1 = 9). So, the null hypothesis is satisfied.


#### Q5

In [8]:
trimat = np.loadtxt("./mstrimat.txt", delimiter="   ")
trimat

array([[ 2., -1.,  0.,  0.,  0.],
       [-1.,  2., -1.,  0.,  0.],
       [ 0., -1.,  2., -1.,  0.],
       [ 0.,  0., -1.,  2., -1.],
       [ 0.,  0.,  0., -1.,  2.]])

In [9]:
import numpy as np

def norm(x):
    """
    Returns norm of a vector, x

    """
    return np.sqrt(x @ x)

def power(A, eps=1e-10):
    """
    Power Iterator with Matrix Deflation
    
    """
    N = A.shape[0]
    eigs = []
    vecs = []
    Ac = np.copy(A)
    lamb = 0

    for i in range(N):
        # Initial guess
        x = np.ones(N)
        while True:
            x_1 = Ac.dot(x) # y_n = A * x_(n-1)
            x_norm = norm(x_1)
            x_1 = x_1 / x_norm # x_n = y_n / ||y_n||

            if(abs(lamb - x_norm) <= eps): # If precision is reached, it returns eigenvalue
                break

            lamb = x_norm
            x = x_1

        eigs.append(lamb)

        # Matrix Deflation: A - Lambda * norm[V] * norm[V]^T
        v = x_1 / norm(x_1)
        R = v * v.T
        R = eigs[i] * R
        vecs.append(x_1)
        Ac = Ac - R

    return eigs, vecs

In [10]:
e, v = power(trimat)
print("First two eigenvalues and eigenvectors:")
for i in range(2):
    print(f"Eigenvalue: {e[i]}\nEigenvector: {v[i]}")
    print()

First two eigenvalues and eigenvectors:
Eigenvalue: 3.7320508074913867
Eigenvector: [ 0.2886764 -0.5        0.577349  -0.5        0.2886764]

Eigenvalue: 3.7347504708527888
Eigenvector: [ 0.29513701 -0.4945395   0.58021519 -0.4945395   0.29513701]



In [11]:
a = c = -1
b = 2
n = 5
K = [1, 2] # Because we are only concerned with the 2 largest

lambda_k = []
v_ik = []
for k in range(2):
    lambda_k.append(b + 2 * np.sqrt(a * c) * np.cos((k + 1) * np.pi / (n + 1)))
    v = np.empty(n)
    for i in range(n):
        v[i] = 2 * (np.sqrt(c / a)) ** (k + 1) * np.sin((i + 1) * (k + 1) * np.pi / (n + 1))
    v_ik.append(v / norm(v))

print("Expected first two eigenvalues and eigenvectors:")
for i in range(2):
    print(f"Eigenvalue: {lambda_k[i]}\nEigenvector: {v_ik[i]}")
    print()

Expected first two eigenvalues and eigenvectors:
Eigenvalue: 3.7320508075688776
Eigenvector: [0.28867513 0.5        0.57735027 0.5        0.28867513]

Eigenvalue: 3.0
Eigenvector: [ 5.00000000e-01  5.00000000e-01  7.07050159e-17 -5.00000000e-01
 -5.00000000e-01]



The first set of eigenvalue and eigenvector clearly matches, while second one doesn't. There are a few reasons which can perhaps explain this:
* The Power Iteration method grows increasingly inaccurate for the subsequent eigenvalues and vectors due to error accumulation.
* Perhaps, instead of taking a norm for the lambda updates, taking a dot product between the updates x_vectors might help decreases the error accumulation.


#### Q6

In [12]:
def jacobi_iter(A, B, init_val, iter_lim, tol):
    """
    # A: N by N np.array() | Contains coefficients of the variables (Matrix, A)
    # B: N by 1 np.array() | Contains the constants on RHS (Vector, B)
    # init_val: np.array() | Contains Initial Values
    # iter_lim: Stores Iteration Limit
    # tol: Tolerance value - how precise does the solution need to be
    """
    CONV_FLAG = False # Convergence Flag
    ITER_LIM = iter_lim
    var = init_val # Vector, X
    
    for i in range(ITER_LIM):
        var_new = np.zeros_like(var) # stores updated values of all variables (Vector, X)

        for j in range(A.shape[0]):
            # Matrix Multiplying all elements, before A's diagonal (in a row) with all corresponding vars (in Vector, X)
            d = np.dot(A[j, :j], var[:j])
            # Matrix Multiplying all elements, after A's diagonal (in a row) with all corresponding vars (in Vector, X)
            r = np.dot(A[j, j + 1:], var[j + 1:])
            # Updating values of vars
            var_new[j] = (B[j] - d - r)/A[j, j]
        
        # Checks, how close the updated values are, to the previous iteration's values and breaks the loop, if close enough (defined by "tol")
        if np.allclose(var, var_new, atol=tol, rtol=0.):
            print("\nSolution converged, after {} iterations.".format(i))
            CONV_FLAG = True
            break

        var = var_new # Storing the new solution
    # If solution is not obtained (no convergence), after ITER_LIM iterations | Note that, this "else" block belongs to the previous "for" statement and not any "if" statement
    else:
        CONV_FLAG = False

    # If Solution converged
    if CONV_FLAG:
        print("SOLUTION: ", var)
        print("ERRORs: ", np.dot(A, var) - B) # Error in the obtained solution
    else:
        print("\nSolution did not converge, after the specified limit of {} iterations.".format(ITER_LIM))


def gauss_seidel(A, B, init_val, iter_lim, tol):
    """
    # A: N by N np.array() | Contains coefficients of the variables (Matrix, A)
    # B: N by 1 np.array() | Contains the constants on RHS (Vector, B)
    # init_val: np.array() | Contains Initial Values
    # iter_lim: Stores Iteration Limit
    # tol: Tolerance value - how precise does the solution need to be
    """
    CONV_FLAG = False # Convergence Flag
    ITER_LIM = iter_lim
    var = init_val # Vector, X

    for i in range(ITER_LIM):
        var_new = np.zeros_like(var) # stores updated values of all variables (Vector, X)

        for j in range(A.shape[0]):
            # Matrix Multiplying all elements, before A's diagonal (in a row) with all corresponding vars (in Vector, X), that now have updated values
            l = np.dot(A[j, :j], var_new[:j]) # Note, the only change from jacobi_iter() is changing "var" to "var_new"
            # Matrix Multiplying all elements, after A's diagonal (in a row) with all corresponding vars (in Vector, X), that do not have updated values yet
            u = np.dot(A[j, j + 1:], var[j + 1:])
            # Updating values of vars
            var_new[j] = (B[j] - l - u) / A[j, j]
        
        # Checks, how close the updated values are, to the previous iteration's values and breaks the loop, if close enough (defined by "tol")
        if np.allclose(var, var_new, atol=tol, rtol=0.):
            print("\nSolution converged, after {} iterations.".format(i))
            CONV_FLAG = True
            break

        var = var_new # Storing the new solution
    # If solution is not obtained (no convergence), after ITER_LIM iterations | Note that, this "else" block belongs to the previous "for" statement and not any "if" statement
    else:
        CONV_FLAG = False

    # If Solution converged
    if CONV_FLAG:
        print("SOLUTION: ", var)
        print("ERRORs: ", np.dot(A, var) - B) # Error in the obtained solution
    else:
        print("\nSolution did not converge, after the specified limit of {} iterations.".format(ITER_LIM))


In [13]:
A = np.array([
    [-2, 0, 0, -1, 0, 0.5],
    [0, 4, 0.5, 0, 1, 0],
    [0, 0.5, 1.5, 0, 0, 0],
    [-1, 0, 0, -2, 0, 1],
    [0, 1, 0, 0, -2.5, 0],
    [0.5, 0, 0, 1, 0, -3.75]
])

b = np.array([-1, 0, 2.75, 2.5, -3, 2])

print("Jacobi Iteration:")
jacobi_iter(A, b, init_val=np.zeros(6), iter_lim=50, tol=1e-6)

print()

print("Gauss Seidel:")
gauss_seidel(A, b, init_val=np.zeros(6), iter_lim=50, tol=1e-6)

Jacobi Iteration:

Solution converged, after 35 iterations.
SOLUTION:  [ 1.49999955 -0.5         2.         -2.50000051  1.         -0.99999973]
ERRORs:  [ 1.54738835e-06  0.00000000e+00  0.00000000e+00  1.73871588e-06
  0.00000000e+00 -1.76005297e-06]

Gauss Seidel:

Solution converged, after 12 iterations.
SOLUTION:  [ 1.49999932 -0.5         2.         -2.49999966  1.         -1.        ]
ERRORs:  [ 1.02103174e-06 -5.63993297e-14  0.00000000e+00 -2.05200301e-10
  0.00000000e+00  0.00000000e+00]
